# Create Sourcing Optimization Function

In [ ]:
def allocate_units_local_for_local(merged_df, target_volume_df, factory_demand_df):
    total_units_required = factory_demand_df['VOLUME'].sum()

    # Initialize allocated_units and local_for_local columns in merged_df
    merged_df['allocated_units'] = 0
    merged_df['local_for_local'] = False  # Initialize as False; will set true in loop if local

    allocation_result = []

    for _, factory in factory_demand_df.iterrows():
        required_units = factory['VOLUME']
        factory_id = factory['factory_id']
        factory_country = factory['factory_country']

        # Determine local_for_local status for each vendor in merged_df
        for i, item in merged_df.iterrows():
            merged_df.at[i, 'local_for_local'] = item['vendor_country'] == factory_country

        # Sort by local_for_local and cost_per_unit within each local and non-local group
        sorted_merged = merged_df.sort_values(by=['local_for_local', 'cost_per_unit'], ascending=[False, True])

        for _, item in sorted_merged.iterrows():
            vendor_id = item['vendor_id']
            target_volume = target_volume_df[target_volume_df['vendor_id'] == vendor_id]

            if not target_volume.empty:
                target_pct = target_volume['target_allocation_pct'].iloc[0]
                acceptable_range = target_volume['acceptable_range_pct'].iloc[0] / 100.0

                min_allocation = (target_pct - acceptable_range) * total_units_required
                max_allocation = (target_pct + acceptable_range) * total_units_required

                potential_allocation = min(required_units, max_allocation - item['allocated_units'])

                if item['allocated_units'] + potential_allocation >= min_allocation:
                    allocated_units = potential_allocation
                else:
                    allocated_units = max(min_allocation - item['allocated_units'], 0)

                sorted_merged.loc[item.name, 'allocated_units'] += allocated_units
                required_units -= allocated_units

                if allocated_units > 0:
                    allocation_result.append({
                        'factory_id': factory_id,
                        'vendor_id': vendor_id,
                        'allocated_units': allocated_units,
                        'local_for_local': item['local_for_local']
                    })

                if required_units <= 0:
                    break

    return pd.DataFrame(allocation_result)
